In [40]:
import polars as pl
import pyodbc

In [50]:
schema_saldos = {
    'DESC_INDICADOR': pl.String,
    'FECHA_CONSULTA': pl.Date,
    'HISTORICO': pl.String,
    'INDICADOR_ID': pl.Int32,
    'INTERMEDIARIO_ID': pl.String,
    'PROGRAMA_DESCRIPCION': pl.String,
    'PROGRAMA_ID': pl.Int32,
    'PROGRAMA_ORIGINAL': pl.Int32,
    'RAZON_SOCIAL': pl.String,
    'TIPO_CREDITO_ID': pl.Int16,
    'Número de registros': pl.Int16,
    'SALDO_CONTINGENTE_MN': pl.Float64
}

path_saldos = "../data/raw/LNTG_SALDOS_SIN_AGRUPAMIENTO_Datos migrados.csv"
path_saldos_bmxt = "../data/raw/LNTG_SALDOS_SIN_AGRUPAMIENTO_BANCOMEXT_Datos migrados.csv"
path_saldos_fianzas = "../data/raw/LNTG_SALDOS_SIN_AGRUPAMIENTO_80686_Datos migrados.csv"

saldos = pl.read_csv(path_saldos, schema_overrides=schema_saldos)
saldos_fianzas = pl.read_csv(path_saldos_fianzas, schema_overrides=schema_saldos)
saldos_bmxt = pl.read_csv(path_saldos_bmxt, schema_overrides=schema_saldos)

saldos_totales = pl.concat([saldos, saldos_bmxt, saldos_fianzas.drop("HISTORICO")])

saldos_totales = (
    saldos_totales.select(
        pl.col("DESC_INDICADOR").alias("Producto"),
        pl.col("FECHA_CONSULTA"),
        pl.col("INDICADOR_ID").alias("Producto ID"),
        pl.col("INTERMEDIARIO_ID"),
        pl.col("PROGRAMA_DESCRIPCION").alias("Programa Descripción (ID)"),
        pl.col("PROGRAMA_ID"),
        pl.col("PROGRAMA_ORIGINAL"),
        pl.col("RAZON_SOCIAL").alias("Razón Social (Intermediario)"),
        pl.col("TIPO_CREDITO_ID"),
        pl.col("SALDO_CONTINGENTE_MN").alias("SUM Saldo_Contingente_Mn")
    )
)


In [51]:
def genera_tpro_clave(df):
    result = df.with_columns(
        pl.when((pl.col("PROGRAMA_ID")>=32000)&(pl.col("PROGRAMA_ID")<=32100))
        .then(pl.col("PROGRAMA_ID"))
        .when((pl.col("PROGRAMA_ID")==3976)&(pl.col("PROGRAMA_ORIGINAL")==31415))
        .then(pl.col("PROGRAMA_ID"))
        .when((pl.col("PROGRAMA_ID")==33366)&(pl.col("PROGRAMA_ORIGINAL")==33842))
        .then(pl.col("PROGRAMA_ID"))
        .when((pl.col("PROGRAMA_ID").is_in([3536, 3537, 3539, 3542,3544, 3545, 3546,3547,3548,3549,3550, 3553, 3555, 3558,3559, 3560, 3564,3566]))&(pl.col("PROGRAMA_ORIGINAL")==3200))
        .then(pl.col("PROGRAMA_ID"))
        .when(pl.col("PROGRAMA_ORIGINAL")==3999)
        .then("PROGRAMA_ID")
        .otherwise(pl.col("PROGRAMA_ORIGINAL")).alias("TPRO_CLAVE")
    )
    return result


In [52]:
saldos_totales = genera_tpro_clave(saldos_totales)

In [53]:
# Agregar archivos .accdb con ´sudo nano /etc/odbc.ini´
dsn = "Catalogos_Dic24"
user = ""
password = ""

conn = pyodbc.connect(f"DSN={dsn};UID={user};PWD={password}")
cursor = conn.cursor()

tipo_credito = pl.read_database(query='SELECT * FROM [TIPO_CREDITO]', connection=conn, schema_overrides={'Tipo_Credito_ID': pl.Int32})

conn.close()

In [54]:
saldos_totales_vf = (
    saldos_totales.join(tipo_credito.select(['Tipo_Credito_ID', 'NR_R']),
                        left_on='TIPO_CREDITO_ID', right_on='Tipo_Credito_ID',
                        how='left')
)

In [ ]:
# Agrup saldo total
(
    saldos_totales_vf
    .group_by([
        "Producto", 
        "Razón Social (Intermediario)", 
        "INTERMEDIARIO_ID", 
        "PROGRAMA_ORIGINAL", 
        "PROGRAMA_ID"
        ]).agg(
            pl.col("SUM Saldo_Contingente_Mn").sum().alias("SALDO_MN")
        )
)

Producto,Razón Social (Intermediario),INTERMEDIARIO_ID,PROGRAMA_ORIGINAL,PROGRAMA_ID,SALDO_MN
str,str,str,i32,i32,f64
"""GARANTIA SECTORIAL""","""BBV BANCOMER""","""10040012""",3999,321292,4.9334e8
"""GARANTIA EMPRESARIAL""","""BANCA AFIRME""","""10040062""",37006,34784,1.3356e6
"""GARANTIA SECTORIAL""","""BANCO DEL BAJIO""","""10040030""",3999,320540,4.3609e6
"""GARANTIA EMPRESARIAL""","""BANCO MERCANTIL DEL NORTE""","""10040072""",35209,34282,448671.71778
"""GARANTIA SECTORIAL""","""BANCO SANTANDER""","""10000144""",3999,321055,495428.598
…,…,…,…,…,…
"""GARANTIA SUBASTA""","""BANCO SANTANDER""","""10000144""",3999,36866,2416.904768
"""GARANTIA SUBASTA""","""BANCO MERCANTIL DEL NORTE""","""10040072""",3999,36614,17091.063888
"""GARANTIA SECTORIAL""","""BANCO MERCANTIL DEL NORTE""","""10040072""",3999,320933,4.0498e8


In [64]:
# Agrup saldo total resto
(
    saldos_totales_vf
    .filter(
        (pl.col("Producto") == "GARANTIAS BANCOMEXT") |
        (pl.col("Producto") == "GARANTIAS SHF/LI FINANCIERO") |
        (pl.col("Producto") == "GARANTIAS BANSEFI") |
        (pl.col("Razón Social (Intermediario)") == "FISO PARA EL AHORRO DE ENERGIA ELECTRICA (FASE II)")
    ).group_by([
        "Producto", 
        "Razón Social (Intermediario)", 
        "INTERMEDIARIO_ID", 
        "PROGRAMA_ORIGINAL", 
        "PROGRAMA_ID"
        ]).agg(
            pl.col("SUM Saldo_Contingente_Mn").sum().alias("SALDO_MN")
        )
)

Producto,Razón Social (Intermediario),INTERMEDIARIO_ID,PROGRAMA_ORIGINAL,PROGRAMA_ID,SALDO_MN
str,str,str,i32,i32,f64
"""GARANTIAS BANCOMEXT""","""BANCOMEXTBANCA AFIRME""","""478439031""",3999,39538,528260.913
"""GARANTIAS BANCOMEXT""","""BANCOMEXT/BANCO SABADELL, S.A.…","""162042417""",3999,390100,4.2e6
"""GARANTIAS BANCOMEXT""","""BANCOMEXTHSBC""","""478809852""",3999,390224,3.8209e7
"""GARANTIAS BANCOMEXT""","""BANCOMEXTBANAMEX""","""468297752""",3999,390192,6.8588e6
"""GARANTIAS BANCOMEXT""","""BANCOMEXT/BAJIO""","""64991057""",3999,32368,1.2884e6
…,…,…,…,…,…
"""GARANTIAS BANCOMEXT""","""BANCOMEXT-BANORTE""","""59639999""",3999,39387,292278.708
"""GARANTIAS BANCOMEXT""","""BANCOMEXT MIFEL""","""468577182""",3999,390088,2.5455e7
"""GARANTIAS BANCOMEXT""","""BANCOMEXT MIFEL""","""468577182""",3999,39825,2.2250e6


In [66]:
# Agrup saldo total automatica
(
    saldos_totales_vf
    .filter(
        (pl.col("Producto") != "GARANTIAS BANCOMEXT") &
        (pl.col("Producto") != "GARANTIAS SHF/LI FINANCIERO") &
        (pl.col("Producto") != "GARANTIAS BANSEFI") &
        (pl.col("Razón Social (Intermediario)") != "FISO PARA EL AHORRO DE ENERGIA ELECTRICA (FASE II)")
    ).group_by([
        "Producto", 
        "Razón Social (Intermediario)", 
        "INTERMEDIARIO_ID", 
        "PROGRAMA_ORIGINAL", 
        "PROGRAMA_ID"
        ]).agg(
            pl.col("SUM Saldo_Contingente_Mn").sum().alias("SALDO_MN")
        )
)

Producto,Razón Social (Intermediario),INTERMEDIARIO_ID,PROGRAMA_ORIGINAL,PROGRAMA_ID,SALDO_MN
str,str,str,i32,i32,f64
"""GARANTIA EMPRESARIAL""","""BANAMEX""","""10000233""",3999,310101,9.1811e8
"""GARANTIA EMPRESARIAL""","""BANSI SA""","""10040060""",3999,310023,5.3915e6
"""GARANTIA SECTORIAL""","""HSBC MEXICO""","""10000171""",3999,321567,1.7815e7
"""GARANTIA SECTORIAL""","""BANCO SANTANDER""","""10000144""",3999,320460,1.6620e6
"""GARANTIA SECTORIAL""","""BANAMEX""","""10000233""",3999,320458,701500.912
…,…,…,…,…,…
"""GARANTIA EMPRESARIAL""","""BANCO DEL BAJIO""","""10040030""",3999,33423,572823.94
"""GARANTIA SECTORIAL""","""BBV BANCOMER""","""10040012""",3999,320658,5.7054e7
"""GARANTIA SECTORIAL""","""BANCA MIFEL""","""10000420""",3999,320360,1.7551e7


In [67]:
# AgrupTaxo saldo total
(
    saldos_totales_vf
    .group_by([
        "Producto"
        ]).agg(
            pl.col("SUM Saldo_Contingente_Mn").sum().alias("SALDO_MN")
        )
)

Producto,SALDO_MN
str,f64
"""EMPRESA MEDIANA""",5.6700e8
"""GARANTIA EMPRESARIAL""",8.5872e10
"""GARANTIAS BANCOMEXT""",1.7804e10
"""GARANTIAS FEFA""",4.3282e8
"""GARANTIA SECTORIAL""",1.8809e10
"""GARANTIA EMERGENCIAS""",4.7684e8
"""GARANTIA EMERGENTES""",4.2845e6
"""GARANTIA FIANZA""",4.8309e9
"""GARANTIA ESPECIAL""",6.3411e7


In [ ]:
# AgrupTaxo saldo total resto
(
    saldos_totales_vf
        .filter(
            (pl.col("Producto") == "GARANTIAS BANCOMEXT") |
            (pl.col("Producto") == "GARANTIAS SHF-LI FINANCIERO") |
            (pl.col("Producto") == "GARANTIAS BANSEFI") |
            (pl.col("Razón Social (Intermediario)") == "FISO PARA EL AHORRO DE ENERGIA ELECTRICA (FASE II)")
        ).group_by([
            "Producto"
            ]).agg(
            pl.col("SUM Saldo_Contingente_Mn").sum().alias("SALDO_MN")
        )
)

Producto,SALDO_MN
str,f64
"""GARANTIAS BANCOMEXT""",1.7804e10


In [70]:
(
    saldos_totales_vf
    .filter(
        (pl.col("Producto") != "GARANTIAS BANCOMEXT") &
        (pl.col("Producto") != "GARANTIAS SHF/LI FINANCIERO") &
        (pl.col("Producto") != "GARANTIAS BANSEFI") &
        (pl.col("Razón Social (Intermediario)") != "FISO PARA EL AHORRO DE ENERGIA ELECTRICA (FASE II)")
        ).group_by([
            "Producto"
            ]).agg(
            pl.col("SUM Saldo_Contingente_Mn").sum().alias("SALDO_MN")
        )
)

Producto,SALDO_MN
str,f64
"""GARANTIA EMPRESARIAL""",8.5872e10
"""GARANTIA EMERGENCIAS""",4.7684e8
"""GARANTIA ESPECIAL""",6.3411e7
"""GARANTIAS FEFA""",4.3282e8
"""GARANTIA EMERGENTES""",4.2845e6
"""GARANTIA SUBASTA""",830523.326002
"""GARANTIA SECTORIAL""",1.8809e10
"""EMPRESA MEDIANA""",5.6700e8
"""GARANTIA FIANZA""",4.8309e9
